# Model training examples

This note-book contains training examples for:  
1. LocMod
2. ConstMod 4
3. SpatBHM 2b
4. GRF on $\mu^0$ values from LocMod (for prior distributions)

## Note on parameter naming

Stan and python use differing indexing conventions. The python names are chosen to reflect the naming conventions in the manuscript, whereas Stan requires the first index to be >1. Therefore, $\mu^0$ is referred to as `mu.1` in the Stan-models and the following parameters are shifted backwards by 1.

## Install libraries:

libraries are installed running:

```py
pip install pystan
pip install nest_asyncio
```

`nest_asyncio` is required for running stan from a Jupyter Notebook.

In [ ]:
# import modules
import numpy as np

# data handling
import pandas as pd

# technical packages
from tqdm import tqdm
import os

# import stan
import stan
import nest_asyncio
nest_asyncio.apply()

# set path to your working directory
dir_path = "./"

# 0. Randomly initialized ConstMod, trained on all data
***

- this fit is used to obtain the initilization values for the further models
- fit result already provided along with the data and remaining code in the repository

In [ ]:
#########################
# Set up and preparation
#########################

# read station data
stat_info = pd.read_csv(f"{dir_path}Data/used_stations.csv", dtype={"Stations_id":str})
z_station = stat_info.Stationshoehe
z_grid = pd.read_csv(f"{dir_path}Data/COSMO-REA6data/COSMO-REA6_HSURF_stations.csv").HSURF.values
stat_ids = [str(s).zfill(5) for s in stat_info.Stations_id]

# make a height predictor vector
elevation = np.array(z_station - z_grid)

# read code
file = f"{dir_path}Stan_code/ConstMod_LocMod.stan"
with open(file) as f:
    code = f.read()

# Read wind gust data
fx_train = pd.read_csv(f"{dir_path}Data/fx_training.csv", index_col='date')
y_train = (fx_train - vmean_train).values.T.flatten()
N = y_train.shape[0]

# read and normalize VMAX
vmax= pd.read_csv(f"{dir_path}Data/vmax_training.csv", index_col='date').values.T.flatten()
pred_train = (pred -vmax.mean())/vmax.std()

# read and normalize VMEAN
vmean = pd.read_csv(f"{dir_path}Data/vmean_training.csv", index_col='date').values.T.flatten()
vmean = (vmean -vmean.mean())/vmean.std()

# make altitude predictor vector
z = np.repeat(elevation, fx_train.shape[0])

# make predictor matrices
Mmu = 4
Msigma =  3
Xmu = np.zeros((Mmu,N))
Xmu[0,:] = 1
Xmu[1,:] = pred_train
Xmu[2,:] = vmean
Xmu[3,:] = z/100
Xsigma = np.zeros((Msigma,N))
Xsigma[0,:] = 1
Xsigma[1,:] = pred_train
Xsigma[2,:] = vmean

# set prior distribution parameters
mu_means = np.array((5,2,-0.8, 0.3))
mu_scales = np.array((2,1,1, 0.1))
sigma_means = np.array((0.5,0.3,-0.2))
sigma_scales = np.array((0.1,0.1,0.1))

# construct data for fit
bl_data = {"N":len(y_train), 
           "y": y_train, 
           "Mmu":Mmu, 
           "Msigma": Msigma,
           "xmu":Xmu.T, 
           "xsigma":Xsigma.T, 
           "mu_means":mu_means, 
           "sigma_means":sigma_means,
           "mu_scales":mu_scales,
           "sigma_scales":sigma_scales}

# build model
seed = np.random.uniform(0,10) # ensure, simulation is actually random by overwriting the default random seed in pystan
bl_mod = stan.build(code, data=bl_data, random_seed=seed)

# fit model
bl_fit = bl_mod.sample(num_chains=1, num_samples=1000, num_warmup=500, save_warmup=False).to_frame()

# save data
file = f"{dir_path}Model_fits/Baseline_optimal/Baseline_optimal_complete_fit.csv"
bl_fit.to_csv(file, index=False)

# 1 LocMod
***
- perform local fits of ConstMod 4 as "*local truth*" for comparison
- With the help of these fits, we can assess, whether the predictors work for each location. We suspect to learn, whether the deficiencies at some locations are caused by the model or whether they can be attributed to issus regarding COSMO-REA6
- initilize with expectation values from a free run training of Baseline_optimal

In [ ]:
########################
# Setup and preparation
########################

# set prior distribution parameters
mu_means = np.array((5,2,-0.8))
mu_scales = np.array((2,1,1))
sigma_means = np.array((0.5,0.3,-0.2))
sigma_scales = np.array((0.1,0.1,0.1))

# read station data
stat_info = pd.read_csv(f"{dir_path}Data/used_stations.csv", dtype={"Stations_id":str})
z_station = stat_info.Stationshoehe
z_grid = pd.read_csv(f"{dir_path}Data/COSMO-REA6data/COSMO-REA6_HSURF_stations.csv").HSURF.values
stat_ids = [str(s).zfill(5) for s in stat_info.Stations_id]

# Read wind gust data
fx_train = pd.read_csv(f"{dir_path}Data/fx_training.csv", index_col='date')
vmax_train = pd.read_csv(f"{dir_path}Data/vmax_training.csv", index_col='date')
vmean_train = pd.read_csv(f"{dir_path}Data/vmean_training.csv", index_col='date')

# standardize VMAX as predictor
vmax_norm = (vmax_train - vmax_train.values.mean())/vmax_train.values.std()

# prepare predictand
y = fx_train - vmean_train

# select initial values for the parameters based on the spatially constant model
MC = pd.read_csv(f"{dir_path}Model_fits/Baseline_optimal/Baseline_optimal_complete_fit.csv")[["mu.1", "mu.2", "mu.3", "sigma.1", "sigma.2", "sigma.3"]]
init = {"mu":MC.mean().values[0:3], "sigma":MC.mean().values[3:6]}
init_list = [] # list of dictionaries, as required by Stan
init_list.append(init)

# read code
file = f"{dir_path}Stan_code/ConstMod_LocMod.stan"
with open(file) as f:
    code = f.read()

# save data
if "LocMod" not in os.listdir(f"{dir_path}Model_fits"):
    os.mkdir(f"{dir_path}Model_fits/LocMod_winter")

####################################
# Model fitting for individual station
####################################

# loop over all stations
for i in tqdm(range(len(stat_ids)), position=0):
    # select training data
    y_train = y[np.array(stat_ids)[i]].values.T.flatten()
    N = y_train.shape[0]
    
    # select and normalize VMAX
    vmax = vmax_train[np.array(stat_ids)[i]].values.T.flatten()
    vmax_pred = (vmax -vmax.mean())/vmax.std()

    #select and noormalize VMEAN
    vmean = vmean_train[np.array(stat_ids)[i]].values.T.flatten()
    vmean_pred = (vmean -vmean.mean())/vmean.std()

    # make altitude predictor vector
    z = np.repeat(elevation[i], fx_train.shape[0])

    # construct predictor matricesm Xmu and Xsigma
    Mmu = 3
    Msigma =  3
    Xmu = np.zeros((Mmu,N))
    Xmu[0,:] = 1
    Xmu[1,:] = vmax_pred
    Xmu[2,:] = vmean_pred
    Xsigma = np.zeros((Msigma,N))
    Xsigma[0,:] = 1
    Xsigma[1,:] = vmax_pred
    Xsigma[2,:] = vmean_pred

    # collect data for fit
    bl_data = {"N":len(y_train), 
               "y": y_train, 
               "Mmu":Mmu, 
               "Msigma": Msigma, 
               "xmu":Xmu.T, 
               "xsigma":Xsigma.T, 
               "mu_means":mu_means, 
               "sigma_means":sigma_means,
               "mu_scales":mu_scales, 
               "sigma_scales":sigma_scales}
    
    # build stan  model
    seed = np.random.uniform(0,10) # ensure that the simulation is random by overwriting the default random seed in pystan
    bl_mod = stan.build(code, data=bl_data, random_seed=seed)
    
    # fit model
    bl_fit = bl_mod.sample(num_chains=1, num_samples=1000, num_warmup=250, save_warmup=False, init=init_list).to_frame()
    
    # save data
    file = f"{dir_path}Model_fits/LocMod/LocMod_{}_fit.csv".format(stat_ids[i])
    bl_fit.to_csv(file, index=False)

# 2. ConstMod 4 training in cross validation
- exclude one station from the data and train the model on the rest
- loop over all stations

In [ ]:
########################
# Setup and preparation
########################

# set prior distribution parameters
mu_means = np.array((5,2,-0.8, 0.3))
mu_scales = np.array((2,1,1, 0.1))
sigma_means = np.array((0.5,0.3,-0.2))
sigma_scales = np.array((0.1,0.1,0.1))

# read station data
stat_info = pd.read_csv(f"{dir_path}Data/used_stations.csv", dtype={"Stations_id":str})
z_station = stat_info.Stationshoehe
z_grid = pd.read_csv(f"{dir_path}Data/COSMO-REA6data/COSMO-REA6_HSURF_stations.csv").HSURF.values
stat_ids = [str(s).zfill(5) for s in stat_info.Stations_id]

# Read wind gust data
fx_train = pd.read_csv(f"{dir_path}Data/fx_training.csv", index_col='date')
vmax_train = pd.read_csv(f"{dir_path}Data/vmax_training.csv", index_col='date')
vmean_train = pd.read_csv(f"{dir_path}Data/vmean_training.csv", index_col='date')

# make a height predictor vector
elevation = np.array(z_station - z_grid)[fit_ind] 

# prepare predictand
y = fx_train - vmean_train

# select initial values for the parameters based on the spatially constant model trained at all stations
MC = pd.read_csv(f"{dir_path}Model_fits/Baseline_optimal/Baseline_optimal_complete_fit.csv")[["mu.1", "mu.2", "mu.3", "mu.4", "sigma.1", "sigma.2", "sigma.3"]]
init = {"mu":MC.mean().values[0:4], "sigma":MC.mean().values[4:7]}

init_list = [] # make a list of dictionaries as required by pystan
init_list.append(init) # 

# read code
file = f"{dir_path}Stan_code/ConstMod_LocMod.stan"
with open(file) as f:
    code = f.read()

####################################
# Model fitting in cross-validation
####################################

# save data
if "Baseline_optimal" not in os.listdir(f"{dir_path}Model_fits"):
    os.mkdir(f"{dir_path}Model_fits/Baseline_optimal")

# start cross validation loop
for i in tqdm(range(len(stat_ids)), position=0):
    # select predictand data
    index = list(range(i)) + list(range(i+1,len(stat_ids)))
    y_train = y[np.array(stat_ids)[index]].values.T.flatten()
    N = y_train.shape[0]

    # select and normalize vmax
    vmax = vmax_train[np.array(stat_ids)[index]].values.T.flatten()
    vmax_pred= (vmax - vmax.mean())/vmax.std()

    # select and normalize vmean
    vmean = vmean_train[np.array(stat_ids)[index]].values.T.flatten()
    vmean_pred = (vmean -vmean.mean())/vmean.std()

    # make altitude predictor vector
    z = np.repeat(elevation[index], fx_train.shape[0])

    # make predictor matrices
    Mmu = 4
    Msigma =  3
    Xmu = np.zeros((Mmu,N))
    Xmu[0,:] = 1
    Xmu[1,:] = vmax_pred
    Xmu[2,:] = vmean_pred
    Xmu[3,:] = z/100
    Xsigma = np.zeros((Msigma,N))
    Xsigma[0,:] = 1
    Xsigma[1,:] = vmax_pred
    Xsigma[2,:] = vmean_pred

    # construct data for fit
    bl_data = {"N":len(y_train), 
               "y": y_train, 
               "Mmu":Mmu, 
               "Msigma": Msigma, 
               "xmu":Xmu.T, 
               "xsigma":Xsigma.T, 
               "mu_means":mu_means, 
               "sigma_means":sigma_means,
               "mu_scales":mu_scales, 
               "sigma_scales":sigma_scales}
    
    # build model
    seed = np.random.uniform(0,10) # make sure, simulation is actually random by circumventing the default random seed set by Stan
    bl_mod = stan.build(code, data=bl_data, random_seed=seed)
    
    # fit model
    bl_fit = bl_mod.sample(num_chains=1, num_samples=1000, num_warmup=250, save_warmup=False, init=init_list).to_frame()
    
    # save data
    file = f"{dir_path}Model_fits/Baseline_optimal/Baseline_optimal_{}_fit.csv".format(stat_ids[i])
    bl_fit.to_csv(file, index=False)

# 3. SpatBHM 2b training in cross validation
***
- spatial parameters are selected via `mu_s` and `sig_s` as binary input vectors of the lenght of the number of covariates for both parameters
- size of prior parameter vectors for GRFs and initialization fields have to be adjusted accordingly
- fitting SpatBHM in cross-validation is computationally expensive and can take up to weeks. Run cell with care.

In [ ]:
# ~~~ which parameters are spatial~(binary vectors)~~~~~~~~~~~~~~~~~~~~~~~~~
mu_s = np.array([1,0,1,0]) 
sigma_s = np.array([0,0,0])
n_spat = int(mu_s.sum() + sigma_s.sum())

#~~~number of covariates for each parameter ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
nxmu = 4
nxsig = 3
# including constant offset

# construct model name
mu_names = [f"mu{i}" for i in range(nxmu)]
sigma_names = [f"sigma{i}" for i in range(nxsig)]
par_string = ""
for pn in range(nxmu):
    if mu_s[pn] == 1:
        par_string += f"mu{pn}_"
for pn in range(nxsig):
    if sigma_s[pn] == 1:
        par_string += f"sigma{pn}_"

model_name = f"SM_{par_string}f"
print(model_name)

##################################
# Read training data
#################################

# read station data
stat_info = pd.read_csv(f"{dir_path}Data/used_stations.csv", dtype={"Stations_id":str})
z_station = stat_info.Stationshoehe
z_grid = pd.read_csv(f"{dir_path}Data/COSMO-REA6data/COSMO-REA6_HSURF_stations.csv").HSURF.values
stat_ids = [str(s).zfill(5) for s in stat_info.Stations_id]

# Read wind gust data
fx_train = pd.read_csv(f"{dir_path}Data/fx_training.csv", index_col='date')
vmax_train = pd.read_csv(f"{dir_path}Data/vmax_training.csv", index_col='date')
vmean_train = pd.read_csv(f"{dir_path}Data/vmean_training.csv", index_col='date')

# make a height predictor vector
elevation = np.array(z_station - z_rea6) 

#####################################
# constant input data for all stations
######################################

fit_ConstMod = pd.read_csv(f"{dir_path}Model_fits/Baseline_optimal/baseline_optimal_complete_fit.csv").mean()

# ~~~ priors ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# spatially constant/GRF mean, Prior is Gaussian. Specify as many as you have regression coefficients for location/scale, there is no default value
mu_prior = np.zeros((nxmu,2))
mu_prior[:,0] = fit_ConstMod.loc[["mu.{}".format(i) for i in np.arange(nxmu)+1]].values # expectation
mu_prior[:,1] = (2,1,0.3,0.1) 

sigma_prior = np.zeros((nxsig,2))
sigma_prior[:,0] = fit_ConstMod.loc[["sigma.{}".format(i) for i in np.arange(nxsig)+1]].values # expectation
sigma_prior[:,1] = (0.2,0.2,0.1) 

# Prior for sills, inverse-gamma(a,b)
sills_prior = np.zeros((n_spat,2))
sills_prior[:,0] = np.repeat(5,n_spat)
sills_prior[:,1] = np.repeat(2,n_spat)

# prior for ranges, inverse-gamma(a,b)
ranges_prior = np.zeros((n_spat,2))
ranges_prior[:,0] = np.repeat(1.05,n_spat)
ranges_prior[:,1] = np.repeat(100,n_spat)

# Prior for f, Gamma-distribution
f_prior = np.array([15,0.15])

#~~~~ create data dictionary for stan model ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# collect data for stan-model
spat_data = {"mu_s":mu_s,
             "sigma_s":sigma_s,
             "mu_prior":mu_prior,
             "sigma_prior":sigma_prior,
             "sills_prior":sills_prior,
             "ranges_prior":ranges_prior,
             "f_prior":f_prior}

###############
# Model code
###############

file = f"{dir_path}Stan_code/SpatBHM.stan"

with open(file) as f:
    code = f.read()

#########################################
# Initialization of the model parameters with 0
#########################################

init = {"f":100, 
        "expect_mu":fit_ConstMod.loc[["mu.1","mu.2","mu.3","mu.4"]].values, 
        "expect_sigma":fit_ConstMod.loc[["sigma.1","sigma.2","sigma.3"]].values, 
        "sills":np.repeat(1,n_spat),
        "ranges":np.repeat(100,n_spat)}

# initialize spatial fields, comment in or change input depending on model version
spat_pars = np.zeros((sum(mu_s)+sum(sigma_s),len(stat_ids)-1))
spat_count = 0
for i in range(nxmu):
    if mu_s[i] ==1:
        spat_pars[spat_count] = np.repeat(init["expect_mu"][i],len(stat_ids)-1).reshape(1,len(stat_ids)-1)
        spat_count += 1
for i in range(nxsig):
    if sigma_s[i] ==1:
        spat_pars[spat_count] = np.repeat(init["expect_sigma"][i],len(stat_ids)-1).reshape(1,len(stat_ids)-1)
        spat_count += 1
    
init.update({"spat_pars": spat_pars})

# append to list to plug it into Stan
init_list = []
init_list.append(init)

In [ ]:
#####################################
# Model training in LOO-cross-validation
#####################################

if model_name not in os.listdir("Model_fits"):
    os.mkdir(f"Model_fits/{model_name}")

for i in tqdm(range(len(stat_ids)),position=0):
    # Updating input data
    index = list(range(i)) + list(range(i+1,len(stat_ids))) # remove one station

    # observations
    y = (fx_train - vmean_train)[np.array(stat_ids)[index]].values.T.flatten()
    N = y.shape[0]

    # stelect and standardize vmax
    vmax_cv = vmax_train[np.array(stat_ids)[index]].values.T.flatten()
    vmax_norm_cv = (vmax_cv-vmax_cv.mean())/vmax_cv.std()

    # select and standardize vmean
    vmean_cv = vmean_train[np.array(stat_ids)[index]].values.T.flatten()
    vmean_norm_cv = (vmean_cv-vmean_cv.mean())/vmean_cv.std()

    # prepare station coordinates for fit (remove verification locations)
    x1 = stat_info.geoLaenge.values[index]
    x2 = stat_info.geoBreite.values[index]
    M = len(x1) # number of stations used for training
    coord = np.zeros((M,2))
    coord[:,0] = x1
    coord[:,1] = x2
    
    # make altitude predictor vector
    z = np.repeat(elevation[index], fx_train.shape[0])
    
    # create predictor matrices
    Mmu = 4
    Msigma =  3
    Xmu = np.zeros((Mmu,N))
    Xmu[0,:] = 1 # constant offset
    Xmu[1,:] = vmax_norm_cv
    Xmu[2,:] = vmean_norm_cv
    Xmu[3,:] = z/100
    Xsigma = np.zeros((Msigma,N))
    Xsigma[0,:] = 1 # constant offset
    Xsigma[1,:] = vmax_norm_cv
    Xsigma[2,:] = vmean_norm_cv
    
    # numbers of data
    M = len(index) # number of stations
    
    # informatio on stations
    z_stat = z_station[index]# station altitude
    z_grid = z_rea6[index] # model topography
    
    # assignment vector to station
    nn = np.repeat(range(1,(M+1)), N/M)  # vector assigning data point to station (for hierarchical model)

    # update model data according to the current CV-iteration
    spat_data.update({"N":N, 
             "M":M, 
             "coord":coord, 
             "z_stat":z_stat, 
             "z_grid":z_grid,
             "y":y,
             "nxmu": Mmu,
             "nxsig":Msigma,
             "Xmu":Xmu,
             "Xsigma":Xsigma,
             "nn":nn})
    
    # build model
    seed = int(np.random.uniform(0,10000)) # overwrite Stan's built-in seed with a random number, so that the simulations are independent
    model = stan.build(code, spat_data, random_seed = seed)

    # fit model
    fit = model.sample(num_chains=1, num_samples=1000, num_warmup=500, init=init_list, max_depth=7).to_frame()
    fit.to_csv(f"{dir_path}Model_fits/{model_name}/{model_name}_{stat_ids[i]}_fit.csv", index=False) # save as file
    del fit # clear dataframe

# 4. Training a Gaussian random field on LocMod parameter estimations

***
- Fit a Gaussian random field to the fitted representations of $\mu^0$ in LocMod;
- We use the *posterior predictive means* at each location
- Aim: obtain a sensible prior for the sill and range parameters of the GRFs, as described in the manuscript in Sect. 4.1.

In [ ]:
## select parameter to train GRF, inside stan naming conventions
coeff = "mu.1"

# read station data
stat_info = pd.read_csv(f"{dir_path}Data/used_stations.csv", dtype={"Stations_id":str})
z_station = stat_info.Stationshoehe
z_grid = pd.read_csv(f"{dir_path}Data/COSMO-REA6data/COSMO-REA6_HSURF_stations.csv").HSURF.values
stat_ids = [str(s).zfill(5) for s in stat_info.Stations_id]

# load parameter data from LocMod fits
df = pd.read_csv(f"Model_fits/LocMod/LocMod_fit_{stat_ids[0]}.csv").mean().iloc[7:]
for s in range(len(stat_ids)-1):
    df = pd.concat((df,pd.read_csv(f"Model_fits/LocMod/LocMod_fit_{stat_ids[s+1]}.csv").mean().iloc[7:]),axis=1)
df=df.T.reset_index(drop=True)

par_values = df["mu.1"].values

# get coordinates
x1 = stat_info.geoLaenge.values[fit_ind]
x2 = stat_info.geoBreite.values[fit_ind]
M = len(x1)
coord = np.zeros((M,2))
coord[:,0] = x1
coord[:,1] = x2

# read Stan code
file = f"{dir_path}Stan_code/GaussRandomField.stn"

with open(file) as f:
    code = f.read()

data = {"N":M, "y":par_values, "coord":coord}

model = stan.build(code, data=data)
fit = model.sample(num_chains=1, num_samples=200, save_warmup=True, num_warmup=200)

fit.to_frame().loc[200:].to_csv(f"{dir_path}Model_fits/GRF/GRF_{coeff}.csv", index =False)